EmDigit Processing Notebook - New version




In [ ]:
#Input the unique id name for the itinerary composed of author initials (SA in case of no known author), date of publication (SD in case of no known date)
#and distinguishing letter in case of multiple editions of same year (i.e. "GH1563A")
itinerary_name = "GM1684"

Installations to be run once before executing the notebook


In [ ]:
#Installs (run once)
!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install -U gspread-dataframe
!pip install numpy

#Packages
import xml.etree.ElementTree as ET
import os
import csv as csv
import pandas as pd
import numpy as np
from pandas.core.arrays.interval import NA
import re
import string
from fuzzywuzzy import fuzz

Mounting google drive so that it has access to Transkribus XML files

In [ ]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Setting parameters for the data

In [ ]:
#Variables for input - note that these will depend upon a given itinerary. Not all books will have margin annotations.
min_maintext_vpos = 100 #The minimum coordinate for main text
max_maintext_vpos = 2250 #The maximum coordinate for main text
#margin_indicator = "—"

Extracting data

In [ ]:
directory = '/content/drive/MyDrive/EmDigitPageFiles/GM1684/497635/GM1684/page'
output_filename = 'page.csv'

In [ ]:
import os
import xml.etree.ElementTree as ET
import pandas as pd

# Function to extract data from multiple XML files and save as a CSV and DataFrame
def extract_text_lines_from_directory(directory, output_csv_file):
    # Define the namespace
    ns = {'ns': 'http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15'}

    # List to store all extracted data
    all_data = []

    # Walk through the directory and its subdirectories
    for subdir, dirs, files in os.walk(directory):
        for filename in files:
            fpath = os.path.join(subdir, filename)

            # Process only XML files
            if fpath.endswith(".xml"):
                try:
                    tree = ET.parse(fpath)
                    root = tree.getroot()

                    # Safely extract relevant metadata from TranskribusMetadata
                    transkribus_metadata = root.find(".//ns:TranskribusMetadata", ns)
                    docId = transkribus_metadata.get('docId') if transkribus_metadata is not None else None
                    pageId = transkribus_metadata.get('pageId') if transkribus_metadata is not None else None
                    pageNr = transkribus_metadata.get('pageNr') if transkribus_metadata is not None else None

                    # Iterate through each TextRegion element
                    for textregion in root.findall(".//ns:TextRegion", ns):
                        region_id = textregion.get("id")
                        region_custom = textregion.get("custom")

                        # Iterate through each TextLine element within the TextRegion
                        for textline in textregion.findall(".//ns:TextLine", ns):
                            line_id = textline.get("id")
                            custom = textline.get("custom")
                            coords = textline.find(".//ns:Coords", ns)
                            points = coords.get("points") if coords is not None else None
                            unicode_elem = textline.find(".//ns:Unicode", ns)
                            content = unicode_elem.text if unicode_elem is not None else None

                            # Store the extracted data in a list
                            all_data.append([docId, pageId, pageNr, region_id, region_custom, line_id, custom, points, content])

                except ET.ParseError as e:
                    print(f"Error parsing {fpath}: {e}")
                except Exception as e:
                    print(f"Error processing {fpath}: {e}")

    # Convert the collected data into a DataFrame
    columns = ["docId", "pageId", "pageNr", "region_id", "region_custom", "line_id", "custom", "points", "content"]
    if all_data:
        extract_df = pd.DataFrame(all_data, columns=columns)
    else:
        # Create an empty DataFrame if no data was collected
        extract_df = pd.DataFrame(columns=columns)

    # Return the DataFrame
    return extract_df

# Example usage
directory = '/content/drive/MyDrive/EmDigitPageFiles/GM1684/497635/GM1684/page'
output_filename = 'page.csv'

extract_df = extract_text_lines_from_directory(directory, output_filename)

# Check the result
print(extract_df)


In [ ]:
extract_df.to_csv('/content/drive/MyDrive/EmDigitPageFiles/GM1684/497635/GM1684/page/output.csv', index=False)
extract_df

In [ ]:
def extract_text_lines_from_directory(directory, output_csv_file):
    # Define the namespace
    ns = {'ns': 'http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15'}

    # List to store all extracted data
    all_data = []

    # Walk through the directory and its subdirectories
    for subdir, dirs, files in os.walk(directory):
        for filename in files:
            fpath = os.path.join(subdir, filename)

            # Process only XML files
            if fpath.endswith(".xml"):
                try:
                    tree = ET.parse(fpath)
                    root = tree.getroot()

                    # Safely extract relevant metadata from TranskribusMetadata
                    transkribus_metadata = root.find(".//ns:TranskribusMetadata", ns)
                    docId = transkribus_metadata.get('docId') if transkribus_metadata is not None else None
                    pageId = transkribus_metadata.get('pageId') if transkribus_metadata is not None else None
                    pageNr = transkribus_metadata.get('pageNr') if transkribus_metadata is not None else None

                    # Iterate through each TextRegion element
                    for textregion in root.findall(".//ns:TextRegion", ns):
                        region_id = textregion.get("id")
                        region_custom = textregion.get("custom")

                        # Iterate through each TextLine element within the TextRegion
                        for textline in textregion.findall(".//ns:TextLine", ns):
                            line_id = textline.get("id")
                            custom = textline.get("custom")
                            coords = textline.find(".//ns:Coords", ns)
                            points = coords.get("points") if coords is not None else None
                            unicode_elem = textline.find(".//ns:Unicode", ns)
                            content = unicode_elem.text if unicode_elem is not None else None

                            # Store the extracted data in a list
                            all_data.append([docId, pageId, pageNr, region_id, region_custom, line_id, custom, points, content])

                except ET.ParseError as e:
                    print(f"Error parsing {fpath}: {e}")
                except Exception as e:
                    print(f"Error processing {fpath}: {e}")

    # Convert the collected data into a DataFrame
    if all_data:
        columns = ["docId", "pageId", "pageNr", "region_id", "region_custom", "line_id", "custom", "points", "content"]
        extract_df = pd.DataFrame(all_data, columns=columns)
        return extract_df
    else:
        # Return an empty DataFrame with the same columns if no data was found
        columns = ["docId", "pageId", "pageNr", "region_id", "region_custom", "line_id", "custom", "points", "content"]
        return pd.DataFrame(columns=columns)

# Example usage
directory = '/content/drive/MyDrive/EmDigitPageFiles/GM1684/497635/GM1684/page'
output_filename = 'page.csv'
extract_df = extract_text_lines_from_directory(directory, output_filename)

In [ ]:
extract_df.to_csv('/content/drive/MyDrive/EmDigitPageFiles/GM1684/497635/GM1684/page/output.csv', index=False)
extract_df

Data Tidying (tidy_df)
This section takes the extract_df (output.csv) and adds columns helpful for sorting, later categorical analysis, and further tidying

In [ ]:
#Optional: Reimport and view result of Text to Data
#extract_df = pd.read_csv("/content/drive/Shareddrives/EmDigIt/Processing/extract_df.csv")
tidy_df = extract_df

#Trim whitespace
tidy_df['content'] = tidy_df['content'].str.replace(r'\s+', ' ').str.strip()
tidy_df

Processing code

In [ ]:
# Function to process each row in the points column
def extract_positions(points):
    # Split the points string into a list of pairs (e.g., "423,260")
    pairs = points.split()

    # Split each pair into horizontal and vertical components and convert them to integers
    hpos = [int(pair.split(',')[0]) for pair in pairs]  # Extract horizontal positions
    vpos = [int(pair.split(',')[1]) for pair in pairs]  # Extract vertical positions

    # Calculate max and min for both horizontal and vertical positions
    max_hpos = max(hpos)
    min_hpos = min(hpos)
    max_vpos = max(vpos)
    min_vpos = min(vpos)

    return pd.Series([max_hpos, min_hpos, max_vpos, min_vpos])

# Apply the function to each row in the "points" column
tidy_df[['max_hpos', 'min_hpos', 'max_vpos', 'min_vpos']] = tidy_df['points'].apply(extract_positions)

# Check the updated DataFrame
tidy_df

Code for additional Columns - same as before


In [ ]:
# Create page, region, and line columns
tidy_df["page"] = tidy_df["pageNr"].astype('int64')
tidy_df['region'] = tidy_df['region_custom'].str.extract(r'readingOrder\s*{index:(\d+);}', expand=False).astype(int) + 1
tidy_df["line"] = tidy_df['custom'].str.extract(r'readingOrder\s*{index:(\d+);}', expand=False).astype(int) + 1

# Sort by page and region
tidy_df = tidy_df.sort_values(['page', 'region'])

# Create page_region_line column
tidy_df['page_region'] = 'P' + tidy_df['page'].astype(str) + 'R' + tidy_df['region'].astype(str)
tidy_df['page_region_line'] = 'P' + tidy_df['page'].astype(str) + 'R' + tidy_df['region'].astype(str) + 'L' + tidy_df['line'].astype(str)

# Select columns
columns = ["docId", "page", "region", "line", "page_region", "page_region_line", "content", "min_vpos", "max_vpos", "min_hpos", "max_hpos"]
tidy_df = pd.DataFrame(tidy_df, columns=columns)

tidy_df


In [ ]:
import re

# Normalize spaces
tidy_df['content'] = tidy_df['content'].str.replace(r'\s+', ' ', regex=True)

# Function to handle p. combinations
def standardize_p_abbr(match):
    following_char = match.group(1).lower()
    return ' P.1' if following_char in ['l', 'i'] else f' P.{following_char}'

# Dictionary of abbreviations and normalizations
abbrevs = {
    'ā': 'an', 'ō': 'on', 'õ': 'on', 'ē': 'en', 'ꝑ': 'per',
    'Gio\\.': "Giovanni", 'Giac\\.': 'Giacomo',
    r'\b[Cc]it tà\b': 'città', r'\b[Cc]a stello\b': 'castello',
    r'\b[Pp]ro vincia\b': 'provincia', r'\b[Pp]rinci pato\b': 'principato',
    'â': 'à', '-': '¬', '=': '¬', '&': 'e',
    r'\.\.': '.', r' \.': '.',
}

# Apply abbreviations
for old, new in abbrevs.items():
    tidy_df['content'] = tidy_df['content'].str.replace(old, new, regex=True)

# Apply p. abbreviation normalization
tidy_df['content'] = tidy_df['content'].str.replace(r'(?i)\s+p\. ?([lLi0-9])', standardize_p_abbr, regex=True)

# Decimal symbols and their replacements
decimals = {
    r'(?<=\d)[\.,]?\*': ".5",
}
for old, new in decimals.items():
    tidy_df['content'] = tidy_df['content'].str.replace(old, new, regex=True)

# Create a DataFrame for short entries at the bottom of the page
cw_df = tidy_df.loc[(tidy_df['content'].str.len() <= 6) & (tidy_df['max_vpos'] >= max_maintext_vpos)]

# Drop these entries from the main DataFrame
tidy_df = tidy_df.drop(cw_df.index)

# Resulting tidy_df contains cleaned content
tidy_df


Saving everything to a CSV file

In [ ]:
# Define the correct directory and output filename
directory = '/content/drive/MyDrive/EmDigitPageFiles/GM1684/497635/GM1684/page'
output_filename = 'tidy_df.csv'

# Full file path
file_path = f"{directory}/{output_filename}"

# Save the DataFrame to the specified location
tidy_df.to_csv(file_path, index=False)


In [ ]:
print(tidy_df.head())



The optimized code processes text data from a DataFrame by aggregating rows based on page_region, cleaning and standardizing text (removing unnecessary separators and hyphens), and splitting concatenated text into individual rows. It adds line numbers and unique identifiers (page_region_line) for each line, organizes the data by page and region, and outputs two structured CSV files: one for aggregated text regions and another for individual text lines.

In [ ]:
import re
import pandas as pd

# Load the tidy_df DataFrame
tidy_df = pd.read_csv("/content/drive/MyDrive/EmDigitPageFiles/GM1684/497635/GM1684/page/tidy_df.csv")

# Make a copy of the DataFrame
tr_df = tidy_df.drop(columns=['page_region_line']).copy()

# Add pipe symbol to the end of each content
tr_df['content'] = tr_df['content'].astype(str) + '|'

# Aggregate data by page_region
agg_functions = {
    'page': 'first',
    'region': 'first',
    'content': ' '.join,
    'max_vpos': 'max',
    'max_hpos': 'max',
    'min_vpos': 'min',
    'min_hpos': 'min',
    'docId': 'first'
}
tr_df = tr_df.groupby('page_region', as_index=False).agg(agg_functions)

# Save the text regions DataFrame to CSV
tr_df.to_csv('/content/drive/MyDrive/EmDigitPageFiles/GM1684/497635/GM1684/page/text_regions.csv', index=False)

# Function to remove separators
def remove_separator(text):
    pattern = r"(?<![0-9]\.)(?<![0-9])(?<!\.)(?<!\—)\|(?!Da|DA|[AÀaà] |[Aa]l |Parte\b|Poste\b|Leghe\b|Viaggio\b|Qui\b|Parti\b)"
    return re.sub(pattern, '', text)

# Apply separator removal and replace hyphens
tr_df['content'] = tr_df['content'].apply(remove_separator).str.replace('¬', '-', regex=False).str.replace('-', '', regex=False)

# Split content based on the pipe separator and explode into new rows
nl_df = tr_df.copy()
nl_df["content"] = nl_df["content"].str.split("|")
nl_df = nl_df.explode('content')

# Trim whitespace and filter non-empty rows
nl_df['content'] = nl_df['content'].str.strip()
nl_df = nl_df[nl_df['content'].str.len() > 0]

# Sort by page and region
nl_df = nl_df.sort_values(['page', 'region']).reset_index(drop=True)

# Add line and page_region_line columns
nl_df['line'] = nl_df.groupby('page_region').cumcount() + 1
nl_df['page_region_line'] = "P" + nl_df["page"].astype(str) + "R" + nl_df["region"].astype(str) + "L" + nl_df['line'].astype(str)

# Export the new line DataFrame to CSV
nl_df.to_csv('/content/drive/MyDrive/EmDigitPageFiles/GM1684/497635/GM1684/page/new_lines.csv', index=False)

nl_df


Followed by automated Tagging

In [ ]:
tg_df = nl_df

The code systematically classifies text lines into predefined categories (page-number, page-header, margin-annotation, chapter-heading, route-header, sum-distance, prose, or location) based on their content, length, and position, ensuring accurate tagging for further analysis.

In [ ]:
import numpy as np

# Define the margin indicator
margin_indicator = "-"  # Replace with the correct margin symbol in your dataset

# Identify page-numbers and page-headers based on their position and content
tg_df.loc[tg_df['content'].str.match(r'^\d+$', na=False) & (tg_df['min_vpos'] <= min_maintext_vpos), 'line_type'] = 'page-number'
tg_df.loc[~tg_df['content'].str.match(r'^\d+$', na=False) & (tg_df['min_vpos'] <= min_maintext_vpos), 'line_type'] = 'page-header'

# Identify page numbers globally if not restricted to the top of the page
tg_df.loc[tg_df['content'].str.match(r'^\d+$', na=False), 'line_type'] = 'page-number'

# Tag marginal annotations or glosses if margin symbols are present
tg_df.loc[tg_df['content'].str.contains(margin_indicator, na=False), 'line_type'] = 'margin-annotation'
regions_with_margin_annotation = tg_df[tg_df['line_type'] == 'margin-annotation']['page_region'].unique()
tg_df.loc[tg_df['page_region'].isin(regions_with_margin_annotation), 'line_type'] = 'margin-annotation'

# Identify chapter headings based on keywords
chapter_patterns = [
    r'^Capitolo', r'^CAPITOLO', r'^ITINERARIO', r'^Parti ', r'^Libro', r'^LE POSTE',
    r'^POSTE', r'^D\'ITALIA', r'^Poste per diverse'
]
for pattern in chapter_patterns:
    tg_df.loc[tg_df['content'].str.contains(pattern, regex=True, na=False), 'line_type'] = 'chapter-heading'

# Identify route headers based on keywords
route_patterns = [
    r'^Poste d', r'^Viaggio ', r'^Altra ', r'^Communicazione ', r'^Strada '
]
for pattern in route_patterns:
    tg_df.loc[tg_df['content'].str.contains(pattern, regex=True, na=False), 'line_type'] = 'route-header'

# Identify sum-distances based on unit patterns
sum_ind = [r'^Nu\.', r'^nu\.', r'^[Pp]oste \d', r'^[Ll]eghe \d', r'^[Mm]iglia \d']
for pattern in sum_ind:
    tg_df.loc[tg_df['content'].str.contains(pattern, regex=True, na=False), 'line_type'] = 'sum-distance'

# Tag long texts as "prose"
tg_df.loc[tg_df['content'].str.len() > 180, 'line_type'] = 'prose'

# Set remaining undefined values to "location"
tg_df['line_type'] = tg_df['line_type'].astype(str).replace('nan', np.nan)
tg_df['line_type'] = tg_df['line_type'].fillna('location')

# Optional: Access rows tagged as "prose" for review
tg_df.loc[tg_df['line_type'] == 'prose'].sample(20)



# Correcting Multiple Pages (concat_df)


In [ ]:
# Set fix_df to tg_df
fix_df = tg_df

# Filter out rows with fix_df['line_type'] in specified categories
filtered_df = fix_df[~fix_df['line_type'].isin(['page-number', 'chapter-heading', 'page-header'])]

# Initialize a list to store the modified rows
modified_rows = []

# Initialize a flag to track whether to skip rows
skip = False

# Iterate through unique pages
for page in filtered_df['page'].unique():
    # Get rows for the current page
    page_rows = filtered_df[filtered_df['page'] == page]
    max_region_row = page_rows.iloc[-1]

    # Get rows for the next page, if any
    next_page_rows = filtered_df[filtered_df['page'] == page + 1]
    min_region_row = next_page_rows.iloc[0] if not next_page_rows.empty else None

    if (
        min_region_row is not None and
        max_region_row['line_type'] in ['location', 'prose'] and
        min_region_row['line_type'] in ['location', 'prose'] and
        not max_region_row['content'].strip().endswith(tuple(string.digits))
    ):
        # Combine content if conditions are met
        combined_content = f"{max_region_row['content']} {min_region_row['content']}".strip()
        combined_row = max_region_row.copy()
        combined_row['content'] = combined_content

        # Add rows to the modified list
        modified_rows.extend(page_rows.iloc[:-1].values.tolist() if not skip else page_rows.iloc[1:-1].values.tolist())
        modified_rows.append(combined_row.values.tolist())

        # Set skip for the next iteration
        skip = True
    else:
        # Add rows to the modified list without combination
        modified_rows.extend(page_rows.iloc[1:].values.tolist() if skip else page_rows.values.tolist())
        skip = False

# Create a new DataFrame with modified rows
modified_df = pd.DataFrame(modified_rows, columns=fix_df.columns)

modified_df #to print the results

In [ ]:
# Re-add previously filtered data
filtered_df_to_add_back = fix_df[fix_df['line_type'].isin(['page-number', 'chapter-heading', 'page-header'])]
concat_df = pd.concat([filtered_df_to_add_back, modified_df], ignore_index=True)

# Sort by page and region
concat_df = concat_df.sort_values(['page', 'region']).reset_index(drop=True)

# Renumber lines within each page_region
concat_df['line'] = concat_df.groupby('page_region').cumcount() + 1

# Add new page_region_line column
concat_df['page_region_line'] = (
    "P" + concat_df["page"].astype(str) +
    "R" + concat_df["region"].astype(str) +
    "L" + concat_df['line'].astype(str)
)

# Reorder the columns
concat_df = concat_df[['docId', 'page_region_line', 'page_region', 'page', 'region', 'min_vpos', 'min_hpos',
                       'max_vpos', 'max_hpos', 'line_type', 'content']]

# Export the DataFrame to a CSV file
output_path = '/content/drive/MyDrive/EmDigitPageFiles/GM1684/497635/GM1684/page/new_lines.csv'
concat_df.to_csv(output_path, index=False)

print(f"DataFrame successfully exported to {output_path}")


# Code to Automate Checks and Fix Common Errors --- check region assignments, validate distance/unit extraction, and correct common errors




In [ ]:
# Load the concat_lines.csv known as new Lines file
concat_df = pd.read_csv('/content/drive/MyDrive/EmDigitPageFiles/GM1684/497635/GM1684/page/new_lines.csv')

# --- 1. Check Region Assignments ---
# Verify that regions are sequential within each page
def check_region_assignments(df):
    invalid_regions = []
    for page in df['page'].unique():
        regions = df[df['page'] == page]['region'].tolist()
        if sorted(regions) != regions:
            invalid_regions.append(page)
    return invalid_regions

invalid_regions = check_region_assignments(concat_df)
if invalid_regions:
    print(f"Invalid region assignments found in pages: {invalid_regions}")
else:
    print("Region assignments look correct.")

# --- 2. Check Distance/Unit Extracts ---
# Validate that distance and unit fields follow expected patterns
def validate_distance_units(df, distance_pattern=r'\d+(?:\.\d+)?', unit_pattern=r'(km|mi|m|yards)'):
    invalid_rows = df[~df['content'].str.contains(f"{distance_pattern} {unit_pattern}", regex=True, na=False)]
    return invalid_rows

distance_issues = validate_distance_units(concat_df)
if not distance_issues.empty:
    print(f"Distance/unit extraction issues found in {len(distance_issues)} rows:")
    print(distance_issues[['page', 'region', 'content']].head())
else:
    print("Distance/unit extraction looks correct.")

# --- 3. Check Line Content Parsing ---
# Ensure line content meets specific formatting expectations
def validate_line_content(df):
    # Example: Ensure no rows have trailing/leading whitespace or inconsistent punctuation
    df['content'] = df['content'].str.strip()
    issues = df[df['content'].str.match(r'.*\s+|[.,!?]{2,}', na=False)]
    return issues

content_issues = validate_line_content(concat_df)
if not content_issues.empty:
    print(f"Line content parsing issues found in {len(content_issues)} rows:")
    print(content_issues[['page', 'region', 'content']].head())
else:
    print("Line content parsing looks correct.")

# --- 4. Replace Common Errors ---
# Define a dictionary of common misspellings or errors and their corrections
corrections = {
    'recieve': 'receive',
    'adress': 'address',
    'teh': 'the',
    'lable': 'label'
}

# Apply corrections
for wrong, correct in corrections.items():
    concat_df['content'] = concat_df['content'].str.replace(rf'\b{wrong}\b', correct, regex=True)

print("Common errors have been replaced.")

# --- Export the corrected DataFrame ---
output_path = '/content/drive/MyDrive/EmDigitPageFiles/GM1684/497635/GM1684/page/concat_lines_checked.csv'
concat_df.to_csv(output_path, index=False)

print(f"Checked and corrected data exported to {output_path}")

# Location Matching - gloc_df and loc_df


Setup and From preceeding and Following text

In [ ]:
import pandas as pd
import os

# --- 1. Load Data ---
file_path = "/content/drive/MyDrive/EmDigitPageFiles/GM1684/497635/GM1684/page/concat_lines_checked.csv"

# Check if the file exists and load it
if os.path.exists(file_path):
    concat_df = pd.read_csv(file_path)
    print(f"File '{file_path}' loaded successfully!")
else:
    print(f"Error: File not found at '{file_path}'.")
    print("Available files in the directory:")
    directory = "/content/drive/MyDrive/EmDigitPageFiles/GM1684/497635/GM1684/page"
    if os.path.exists(directory):
        print(os.listdir(directory))
    else:
        print(f"Directory not found: {directory}")
    raise FileNotFoundError(f"File not found: {file_path}")

# --- 2. Filter Rows for "location" ---
loc_df = concat_df[concat_df['line_type'] == 'location'].copy()

# Clean the content column
loc_df['cleaned'] = (
    loc_df['content']
    .fillna('')  # Handle NaN values
    .str.lower()  # Convert to lowercase
    .str.replace(r'\s([lL]|[Pp])\.\d+|\s\.\d+$', '', regex=True)  # Remove trailing page/line numbers
    .str.replace(r'^\d+', '', regex=True)  # Remove digits at the beginning
    .str.strip()  # Trim whitespace
)

# Remove rows with empty content after cleaning
loc_df = loc_df[loc_df['cleaned'].str.len() > 0]

# --- 3. Extract Text After Prepositions ---
preps = ['A', 'Alla', 'Al', 'Alle', 'a', 'à', 'alle', 'al', 'a la', 'ala', 'alas',
         'all\'', 'alla', 'allas', 'ale', 'ad', 'allo', 'auff', 'da', 'zu', 'zur', 'zum']

def extract_text_after_preps(text, prepositions):
    """Extract text after a preposition."""
    for prep in prepositions:
        match = re.search(rf'\b{prep}\b\s+(.+)', text)
        if match:
            return match.group(1).strip()
    return text

loc_df['cleaned'] = loc_df['cleaned'].apply(lambda x: extract_text_after_preps(x, preps))

# --- 4. Keep Text Before First Punctuation ---
def keep_before_punctuation(text):
    """Keep only the text before the first punctuation."""
    match = re.match(r'^([^\.!?;,]*)', text)
    return match.group(1).strip() if match else text

loc_df['cleaned'] = loc_df['cleaned'].apply(keep_before_punctuation)

# --- 5. Export Cleaned Data ---
output_path = "/content/drive/MyDrive/EmDigitPageFiles/GM1684/497635/GM1684/page/output2.csv"
loc_df.to_csv(output_path, index=False)
print(f"File saved to: {output_path}")



Instructions and 0ther Non-location data

In [ ]:
import pandas as pd
import re

# Following Text: Keeps only the text that precedes the details provided in the dictionary below

details = [
    'per', 'e per', 'per', 'e', 'ò', 'o', '\&', 'p', 'miglia', 'm', 'mi', 'leg', 'l', 'le', 'leghe', 'leghi', '\d+',
    'città', 'citta', 'metropoli', 'villa', 'uilla', 'villagio', 'villaggio', 'uillagio', 'borgo', 'castello',
    'castellano', 'hosteria', 'osteria', 'capo', 'corte', 'principato', 'provincia', 'chiesa', 'fiume', 'stato',
    'montagna', 'grosso', 'bellissima', 'già', 'ove', "questo", 'havete', 'che', 'vicina', 'vicino', 'dietro',
    'dietra', 'come'
]

for i in loc_df.index:
    for detail in details:
        if re.search(fr"\b{detail}\b|\d", loc_df.loc[i, 'cleaned']):
            match = re.search(fr"\b{re.escape(detail)}\b", loc_df.loc[i, 'cleaned'])
            if match:
                split_point = match.start()
                loc_df.loc[i, 'cleaned'] = loc_df.loc[i, 'cleaned'][:split_point].strip()
                break

# Preceding Text: Keeps only the text that follows the instructions provided in the dictionary below

instr = [
    "passate", "passarete", "si ascende", "ascenderete", "si imbarca", "se imbarca", "volete", "volendo", "vedrete",
    "trovate", "cominciate", "si passa", "passare", "qui", "qua", "quà", "quella", "quello", "quelle", "quelli",
    "questo", "questa", "questi", "queste", "poi", "volendo", 'gran', "guardatevi", "guardate", "imbarcatevi",
    "ove", "pigliate", "da detta", "da questa", "poste", "viaggio", "quì", "sono", "una"
]

for i in loc_df.index:
    for instruction in instr:
        if instruction in loc_df.loc[i, 'cleaned']:
            match = re.search(rf'\b{instruction}\b.*', loc_df.loc[i, 'cleaned'])
            if match:
                loc_df.loc[i, 'cleaned'] = ''
                break

# Remove short entries: Set to blank if consisting of two characters or fewer


for i in loc_df.index:
    if len(loc_df.loc[i, 'cleaned']) <= 2:
        loc_df.loc[i, 'cleaned'] = ''

# Save the final cleaned DataFrame to a CSV file

loc_df.to_csv('/content/drive/MyDrive/EmDigitPageFiles/GM1684/497635/GM1684/page/locations_extract.csv', index=False)

# **Code to avoid hand checking locations extract **


In [ ]:

# Load the locations_extract.csv file
file_path = '/content/drive/MyDrive/EmDigitPageFiles/GM1684/497635/GM1684/page/locations_extract.csv'
loc_df = pd.read_csv(file_path)

# Define a function to check and flag potential issues
def check_and_flag_locations(row):
    # Flag word fragments or entries with special characters
    if re.search(r'\b\w{1,2}\b', row) or re.search(r'[^a-zA-Z\s]', row):
        return True
    return False

# Apply the flagging function to the 'cleaned' column
loc_df['flag'] = loc_df['cleaned'].apply(lambda x: check_and_flag_locations(x) if isinstance(x, str) else False)

# Filter rows that are flagged for manual inspection
flagged_df = loc_df[loc_df['flag'] == True]

# Function to clean common errors
def correct_common_errors(text):
    corrections = {
        'uillagio': 'villaggio',
        'citta': 'città',
        'metropoli ': 'metropoli',
        # Add more common corrections as needed
    }
    for error, correction in corrections.items():
        text = re.sub(rf'\b{error}\b', correction, text, flags=re.IGNORECASE)
    return text

# Apply the correction function to the 'cleaned' column
loc_df['cleaned'] = loc_df['cleaned'].apply(lambda x: correct_common_errors(x) if isinstance(x, str) else x)

# Save flagged rows for manual review
flagged_file_path = '/content/drive/MyDrive/EmDigitPageFiles/GM1684/497635/GM1684/page/flagged_locations.csv'
flagged_df.to_csv(flagged_file_path, index=False)

# Save the corrected DataFrame back to CSV
corrected_file_path = '/content/drive/MyDrive/EmDigitPageFiles/GM1684/497635/GM1684/page/corrected_locations_extract.csv'
loc_df.to_csv(corrected_file_path, index=False)

print("Flagged rows saved to:", flagged_file_path)
print("Corrected locations file saved to:", corrected_file_path)


Matching to Gazzeteer ---- missing Gazzater Files



In [ ]:
import pandas as pd
import re
from fuzzywuzzy import fuzz

# Load the gazetteer CSV from the correct directory
gaz_df = pd.read_csv("/content/drive/MyDrive/EmDigitPageFiles/GM1684/497635/GM1684/page/gazetteer.csv")

# Reimport and refilter data if gazetteer changes
reimport_df = pd.read_csv("/content/drive/MyDrive/EmDigitPageFiles/GM1684/497635/GM1684/page/full_test_df.csv")
locations_filter = reimport_df['line_type'] == 'location'
loc_df = reimport_df[locations_filter].copy()

# Display the DataFrame
print(loc_df)

# Split the Location_Name field based on the pipe separator
gaz_df["Location_Name"] = gaz_df["Location_Name"].str.split("|")
gaz_df = gaz_df.explode('Location_Name')

# Create the new geolocated dataframe based on loc_df
gloc_df = loc_df.copy()

# Create a dictionary to map routes to their associated states
route_state_dict = gloc_df.groupby('route_description')['state'].apply(lambda x: x.unique()).to_dict()

# Function to find fuzzy matches with a threshold for accuracy
def find_fuzzy_match_with_state(row, route_state_dict, threshold=70):
    if pd.isna(row['cleaned']):
        return None

    # Get matching states for the current route
    matching_states = route_state_dict.get(row['route_description'], [])

    # If no matching states, return None
    if len(matching_states) == 0:
        return None

    # Filter gazetteer entries based on matching states
    filtered_gaz_df = gaz_df[gaz_df['state'].isin(matching_states)]

    # Check for an exact match
    exact_match = filtered_gaz_df[filtered_gaz_df['Location_Name'] == row['cleaned']]
    if not exact_match.empty:
        return exact_match['id'].values[0]

    # Fuzzy matching
    max_ratio = -1
    best_match = None
    for index, gaz_row in filtered_gaz_df.iterrows():
        ratio = fuzz.token_set_ratio(row['cleaned'], gaz_row['Location_Name'])
        if ratio > threshold and ratio > max_ratio:
            max_ratio = ratio
            best_match = gaz_row['id']

    return best_match

# Function to handle user input and apply fuzzy matching
def process_fuzzy_matching(gloc_df, route_state_dict):
    user_input = input("Enter 'all' to run on all rows or 'missing' to run on rows with missing match_id: ").strip().lower()

    if user_input == 'all':
        gloc_df['id'] = gloc_df.apply(lambda row: find_fuzzy_match_with_state(row, route_state_dict), axis=1)
    elif user_input == 'missing':
        missing_rows = gloc_df['id'].isnull()
        gloc_df.loc[missing_rows, 'id'] = gloc_df[missing_rows].apply(lambda row: find_fuzzy_match_with_state(row, route_state_dict), axis=1)
    else:
        print("Invalid input. Please enter 'all' or 'missing'.")

# Apply fuzzy matching
def main():
    process_fuzzy_matching(gloc_df, route_state_dict)

    # Error checkers: Ensure no duplicates or empty IDs
    if gloc_df['id'].isnull().any():
        print("Warning: Some rows still have missing IDs. Please review.")
    if gloc_df.duplicated(subset=['id']).any():
        print("Warning: Duplicate IDs found. Consider resolving conflicts.")

    # Save the output to the correct directory
    output_path = "/content/drive/MyDrive/EmDigitPageFiles/GM1684/497635/GM1684/page/geolocated.csv"
    gloc_df.to_csv(output_path, index=False)
    print(f"Geolocated data saved to: {output_path}")

# Execute the script
if __name__ == "__main__":
    main()



Levenshtein Match - Final option and All methods

In [ ]:
import Levenshtein

# Set the threshold for fuzzy matching
threshold = 0.8  # Adjust this value to your desired threshold (between 0 and 1)

# Load the data
gloc_df = pd.read_csv("/content/drive/MyDrive/EmDigitPageFiles/GM1684/497635/GM1684/page/geolocated.csv")
gaz_df = pd.read_csv("/path/to/gaz_df.csv")  # Replace with the actual path to gaz_df

# Drop unnecessary columns in gloc_df
gloc_df = gloc_df.loc[:, :'id']

# Reset indices
gloc_df.reset_index(drop=True, inplace=True)
gaz_df.reset_index(drop=True, inplace=True)

# Function to find fuzzy match based on the threshold
def find_fuzzy_match(row, gaz_df, threshold):
    max_ratio = -1
    best_match = None
    for _, gaz_row in gaz_df.iterrows():
        ratio = Levenshtein.ratio(row['cleaned'], gaz_row['Location_Name'])
        if ratio > threshold and ratio > max_ratio:
            max_ratio = ratio
            best_match = gaz_row['new_id']
    return best_match

# Apply fuzzy matching to gloc_df
gloc_df['match_id'] = gloc_df.apply(find_fuzzy_match, axis=1, args=(gaz_df, threshold))

# Merge gloc_df with gaz_df based on the match_id and new_id columns
merged_df = gloc_df.merge(
    gaz_df[['geoname', 'id', 'geonameId', 'state', 'country_code', 'Location_Lat', 'Location_Lng']],
    how='left', left_on='match_id', right_on='new_id'
)

# Drop duplicates
merged_df = merged_df.drop_duplicates()

# Export the merged dataframe to CSV
merged_df.to_csv('/content/drive/MyDrive/EmDigitPageFiles/GM1684/497635/GM1684/page/geolocated_merged.csv', index=False)

Gazzetteer Adjustment to reflect new locations - both matched and unmatched

In [ ]:

# Re-import geolocated.csv, dropping unnecessary columns after 'match_id'
gloc_df = pd.read_csv("/content/drive/MyDrive/EmDigitPageFiles/GM1684/497635/GM1684/page/geolocated_merged.csv").loc[:, :'id']

# Load gazetteer CSV
new_gaz_df = pd.read_csv("/content/drive/Shareddrives/EmDigIt/Processing/gazetteer.csv")

# Efficiently map 'cleaned' values to 'Location_Name' where 'id' matches 'match_id'
new_gaz_df.set_index('id', inplace=True)  # Set index for faster lookups
gloc_df.set_index('id', inplace=True)

# Update Location_Name by appending cleaned values where match_id is found
new_gaz_df.loc[gloc_df.index.intersection(new_gaz_df.index), 'Location_Name'] += '|' + gloc_df['cleaned']

# Reset index after modifications
new_gaz_df.reset_index(inplace=True)
gloc_df.reset_index(inplace=True)

# Create a boolean mask for rows with missing match_id
mask = gloc_df['match_id'].isna() | gloc_df['match_id'].isnull() | (gloc_df['match_id'] == '')

# Filter gloc_df to extract new unmatched locations
filtered_gloc_df = gloc_df.loc[mask, ['cleaned']].drop_duplicates().dropna().rename(columns={'cleaned': 'Location_Name'})
filtered_gloc_df['Location_Name_Standardized'] = filtered_gloc_df['Location_Name']

# Define structure for new entries with default empty values
new_rows = pd.DataFrame({
    'id': [''] * len(filtered_gloc_df),
    'Location_Name': filtered_gloc_df['Location_Name'],
    'Location_Name_Standardized': filtered_gloc_df['Location_Name_Standardized'],
    'geonameId': [''] * len(filtered_gloc_df),
    'adminName1': [''] * len(filtered_gloc_df),
    'Location_Lat': [''] * len(filtered_gloc_df),
    'Location_Lng': [''] * len(filtered_gloc_df),
    'geoname': [''] * len(filtered_gloc_df),
    'state': [''] * len(filtered_gloc_df),
    'country_code': [''] * len(filtered_gloc_df),
    'Flag': [''] * len(filtered_gloc_df)
})

# Append the new rows to new_gaz_df in one go (avoids slow row-by-row appends)
new_gaz_df = pd.concat([new_gaz_df, new_rows], ignore_index=True)

# Sort and reset index once at the end (instead of multiple times)
new_gaz_df.sort_values('Location_Name_Standardized', inplace=True)
new_gaz_df.reset_index(drop=True, inplace=True)

# Renumber new_id with index values
new_gaz_df['newest_id'] = new_gaz_df.index + 1

# Corrected file path for exporting the new gazetteer
new_gaz_df.to_csv("/content/drive/MyDrive/EmDigitPageFiles/GM1684/497635/GM1684/page/new_gaz.csv", index=False)


Final Output


In [ ]:
import pandas as pd
import os
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import set_with_dataframe

# ----------------------------- STEP 1: IMPORT DATA -------------------------------- #

# Import the CSV files into dataframes
geolocated_df = pd.read_csv("/content/drive/Shareddrives/EmDigIt/Processing/geolocated.csv")
final_df = pd.read_csv("/content/drive/Shareddrives/EmDigIt/Processing/full_test_df.csv")

# -------------------------- STEP 2: MERGE GEOLOCATED DATA ------------------------- #

# Merge final_df with geolocated_df using 'page_region_line', prioritizing new values
merged_df = pd.merge(final_df, geolocated_df, on='page_region_line', how='left', suffixes=('_final', '_geolocated'))

# Efficiently update values only where geolocated_df has non-null data
for col in geolocated_df.columns:
    if col != 'page_region_line':  # Avoid modifying the key column
        merged_df[col] = merged_df[f'{col}_geolocated'].combine_first(merged_df[f'{col}_final'])

# Drop redundant columns with suffixes
merged_df = merged_df[final_df.columns]

# Save the merged dataframe
merged_df.to_csv('/content/drive/Shareddrives/EmDigIt/Processing/new_final.csv', index=False)

# --------------------------- STEP 3: MERGE WITH GAZETTEER -------------------------- #

# Import the gazetteer dataframe
gaz_df = pd.read_csv("/content/drive/Shareddrives/EmDigIt/Processing/gazetteer.csv")

# Merge on 'id', keeping values from gaz_df where available
final_merged_df = pd.merge(merged_df, gaz_df, on='id', how='left', suffixes=('', '_gaz'))

# Efficiently update final_merged_df with gaz_df values where available
for col in gaz_df.columns:
    if col != 'id' and f'{col}_gaz' in final_merged_df.columns:
        final_merged_df[col] = final_merged_df[f'{col}_gaz'].combine_first(final_merged_df[col])

# Drop redundant '_gaz' columns
final_merged_df = final_merged_df[merged_df.columns]

# Save the final merged dataframe
final_merged_df.to_csv('/content/drive/Shareddrives/EmDigIt/Processing/final_merge.csv', index=False)

# -------------------------- STEP 4: CLEAN & ORGANIZE DATA ------------------------- #

# Optional: reimport concat_lines for additional processing
# concat_df = pd.read_csv("/content/drive/Shareddrives/EmDigIt/Processing/clean_df.csv")

# Sort final_df to maintain order
final_df = final_merged_df.sort_values(['page', 'region', 'line']).drop_duplicates()

# Assign route descriptions efficiently
final_df['route_description'] = ''
route_header_mask = final_df['line_type'] == 'route-header'
final_df.loc[route_header_mask, 'route_description'] = final_df.loc[route_header_mask, 'content']
final_df['route_description'].fillna(method='ffill', inplace=True)  # Fill down until 'sum-distance'

# Save the final cleaned dataframe
final_df.to_csv('/content/drive/Shareddrives/EmDigIt/Processing/final.csv', index=False)

# -------------------------- STEP 5: UPLOAD TO GOOGLE SHEETS ---------------------- #

# Authenticate Google Colab with user's credentials
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Create a new Google Sheets document
sh = gc.create(itinerary_name)

# File-to-Sheet Mapping
file_mappings = {
    'tidy_alto.csv': 'tidied alto',
    'text_regions.csv': 'text regions',
    'new_lines.csv': 'new lines',
    'tagged_lines.csv': 'tagged',
    'concat_lines.csv': 'concatenated',
    'geolocated.csv': 'geolocated',
    'gazetteer.csv': 'gazetteer',
    'final.csv': 'final'
}

# Upload each CSV file to its corresponding Google Sheets tab
for file_name, sheet_name in file_mappings.items():
    file_path = os.path.join("/content/drive/Shareddrives/EmDigIt/Processing", file_name)
    if os.path.exists(file_path):  # Ensure file exists before attempting to read
        df = pd.read_csv(file_path)
        worksheet = sh.add_worksheet(title=sheet_name, rows=df.shape[0] + 1, cols=df.shape[1])
        set_with_dataframe(worksheet, df)

# Print Google Sheets link
print(f"Google Sheets document created: https://docs.google.com/spreadsheets/d/{sh.id}")
